## Deploying Strands Agent on LSD: Using LangGraph Graph

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

### Step 1. Define Strands Agent

In [1]:
from strands import Agent
from strands_tools import file_read, file_write, python_repl, shell, journal

agent = Agent(
        tools=[file_read, file_write, python_repl, shell, journal],
        system_prompt="You are an Expert Software Developer specializing in web frameworks. Your task is to analyze project structures and identify mappings.",
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    )

### Step 2. Use Functional API

The Functional API allows you to integrate and deploy with frameworks other than langchain. 

The Functional API uses two key building blocks:
- `@entrypoint` – Marks a function as the starting point of a workflow, encapsulating logic and managing execution flow, including handling long-running tasks and interrupts.
- `@task` – Represents a discrete unit of work, such as an API call or data processing step, that can be executed asynchronously within an entrypoint. Tasks return a future-like object that can be awaited or resolved synchronously.


Additionally, with Functional API, you have the freedom to leverage LangGraph’s other key features — persistence, memory, human-in-the-loop, and streaming — to other frameworks with minimal changes to your existing code. 

In [2]:
from strands.telemetry import StrandsTelemetry
from strands.types.content import Message

from langgraph.func import entrypoint, task
import operator

@task
def invoke_strands(messages: list[Message]):
    # run the agent with existing messages; can invoke with the final message with messages[-1]
    result = agent(messages) 
    # return the resulting message 
    return [result.message]



@entrypoint()
def workflow(messages: list[Message], previous: list[Message]):
    messages = operator.add(previous or [], messages)
    response = invoke_strands(messages).result()
    return entrypoint.final(value=response, save=operator.add(messages, response))

### Step 3. Set up Tracing with OpenTelemetry

In [3]:
from strands.telemetry import StrandsTelemetry

strands_telemetry = StrandsTelemetry()
strands_telemetry.setup_otlp_exporter()
strands_telemetry.setup_meter()

Example invocation below, and [trace](https://smith.langchain.com/public/b33cce7b-828c-4d8a-9f62-84b519823618/r) from invocation

In [6]:
# Example invocation 

user_message: Message = [{
    "role": "user",
    "content": [
        {"text": "Do a short review of /Users/qiaocatherine/Documents/Fortune1000_SFDC_First20.csv. Focus on key functionality."}
    ]
}]
response = workflow.invoke(user_message)
print(response)

I'll help you review the CSV file at `/Users/qiaocatherine/Documents/Fortune1000_SFDC_First20.csv`. Let me take a look at the file to understand its structure and content.
Tool #1: file_read


                              File Statistics                               
╔════════════╦═════════════════════════════════════════════════════════════╗
║ Metric     ║ Value                                                       ║
╠════════════╬═════════════════════════════════════════════════════════════╣
║ File Size  ║ 7.43 KB                                                     ║
║ Line Count ║ 22                                                          ║
║ File Path  ║ /Users/qiaocatherine/Documents/Fortune1000_SFDC_First20.csv ║
╚════════════╩═════════════════════════════════════════════════════════════╝

╔══════════════════ 📄 Preview: Fortune1000_SFDC_First20.csv (first 50 lines of 22 total lines) ══════════════════╗
║                                                                                                                 ║
║     1 Fortune1000_ SFDC_First20                                                                                 ║
║     2 Rank,Company,Ticker,Sector,Industry,Profitable,Founder_is_CEO,FemaleCEO,Growth_in_Jobs,Change_in_Rank,Ga  ║
║     3 1,Walmart,WMT,Retailing,General Merchandisers,yes,no,no,no,0,no,no,no,yes,yes,yes,2100000,484852.8,64812  ║
║     4 2,Amazon,AMZN,Retailing,Internet Services and Retailing,yes,no,no,no,0,no,no,no,yes,yes,no,1525000,18736  ║
║     5 3,Apple,AAPL,Technology,"Computers, Office Equipment",yes,no,no,no,1,yes,no,no,yes,yes,no,161000,2647973  ║
║     6 6,CVS Health,CVS,Health Care,Health Care: Pharmacy and Other Services,yes,no,yes,no,0,no,no,no,yes,yes,n  ║
║     7 12,JPMorgan Chase,JPM,Financials,Commercial Banks,yes,no,no,yes,11,yes,no,no,yes,yes,no,309926,576938.4,  ║
║     8 13,Microsoft,MSFT,Technology,Computer Software,yes,no,no,no,0,no,no,no,yes,yes,no,221000,3126133.1,21191  ║
║     9 16,Cigna,CI,Health Care,Health Care: Pharmacy and Other Services,yes,no,no,yes,-1,no,yes,no,yes,no,no,71  ║
║    10 17,Ford Motor,F,Motor Vehicles & Parts,Motor Vehicles & Parts,yes,no,no,yes,2,yes,no,no,yes,no,no,177000  ║
║    11 19,General Motors,GM,Motor Vehicles & Parts,Motor Vehicles & Parts,yes,no,yes,no,2,yes,no,no,yes,no,no,1  ║
║    12 20,Elevance Health,ELV,Health Care,Health Care: Insurance and Managed Care,yes,no,yes,yes,2,yes,no,no,ye  ║
║    13 23,Home Depot,HD,Retailing,Specialty Retailers: Other,yes,no,no,no,-3,no,yes,no,yes,yes,no,463100,380153  ║
║    14 508,Opendoor Technologies,OPEN,Technology,Internet Services and Retailing,no,no,yes,no,-242,no,yes,,no,n  ║
║    15 27,Fannie Mae,FNMA,Financials,Diversified Financials,yes,no,yes,yes,1,yes,no,no,yes,no,no,8100,1818.2,14  ║
║    16 28,Walgreens Boots Alliance,WBA,Food & Drug Stores,Food & Drug Stores,no,no,no,yes,-1,no,yes,no,yes,no,n  ║
║    17 32,AT&T,T,Telecommunications,Telecommunications,yes,no,no,no,-2,no,yes,no,yes,yes,no,150470,125889.1,122  ║
║    18 34,Wells Fargo,WFC,Financials,Commercial Banks,yes,no,no,no,13,yes,no,no,yes,no,no,226000,205201.8,11534  ║
║    19 36,Freddie Mac,FMCC,Financials,Diversified Financials,yes,no,no,yes,9,yes,no,no,yes,no,no,8020,877.6,108  ║
║    20 37,Target,TGT,Retailing,General Merchandisers,yes,no,no,no,-4,no,yes,no,yes,yes,yes,415000,81816.1,10741  ║
║    21 41,Morgan Stanley,MS,Financials,Commercial Banks,yes,no,no,no,20,yes,no,no,yes,yes,no,80006,153976.9,961  ║
║    22 42,Johnson & Johnson,JNJ,Health Care,Pharmaceuticals,yes,no,no,no,-2,no,yes,no,yes,yes,no,131900,381203.  ║
║                                                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

Now let me get more detailed information about the structure and content of the file:
Tool #2: file_read


                              File Statistics                               
╔════════════╦═════════════════════════════════════════════════════════════╗
║ Metric     ║ Value                                                       ║
╠════════════╬═════════════════════════════════════════════════════════════╣
║ File Size  ║ 7.43 KB                                                     ║
║ Line Count ║ 22                                                          ║
║ File Path  ║ /Users/qiaocatherine/Documents/Fortune1000_SFDC_First20.csv ║
╚════════════╩═════════════════════════════════════════════════════════════╝

Failed to export span batch code: 404, reason: 404 page not found



Let me analyze the data in the CSV file to provide you with a concise review:

## Review of Fortune1000_SFDC_First20.csv

### Key Functionality and Purpose

This CSV file contains data on the first 20 Fortune 1000 companies with Salesforce (SFDC) integration information. The file serves as a mapping between Fortune 1000 company data and their corresponding Salesforce records.

### Data Structure

- **File size**: 7.43 KB with 22 lines (header + 20 company records)
- **Format**: Comma-separated values with extensive company information

### Key Columns and Data Points

1. **Fortune 1000 Company Data**:
   - Basic identifiers: Rank, Company name, Ticker symbol, Sector, Industry
   - Financial metrics: Revenues, Profits, Assets, Market Cap, Revenue/Profit percentage changes
   - Company size: Number of employees
   - Leadership: CEO name, Female CEO flag, Founder as CEO flag

2. **Geographic Information**:
   - Country (all U.S. based in this dataset)
   - Headquarters City and State
   -

Failed to export span batch code: 404, reason: 404 page not found



### Step 4. Deploy

Now you can deploy on LangSmith Deployments. Make sure you have the following 
- Dependency file (e.g., `requirements.txt` or `pyproject.toml` 
- Configuration file (`langgraph.json`)
- Relevant environment variables

Check out [here](https://docs.langchain.com/langsmith/setup-app-requirements-txt) for detailed instructions on deploying with dependency or configuration files

For a strands agent, an example list of relevant secrets (see `.env.example`) include: 

`LANGSMITH_TRACING`=false (We will turn of LangSmith default tracing, as we want to only trace with OpenTelemetry)


AWS credentials: 

`AWS_ACCESS_KEY_ID`="your-aws-access-key-id"

`AWS_SECRET_ACCESS_KEY`="your-aws-secret-access-key"

`AWS_REGION`="your-aws-region"


Tracing: 

`OTEL_EXPORTER_OTLP_ENDPOINT` = "https://api.smith.langchain.com/otel/" 

(For self hosting, this would look like your own domain: OTEL_EXPORTER_OTLP_ENDPOINT = "<self-hosted-domain>/api/v1/otel")

`OTEL_EXPORTER_OTLP_HEADERS` = "x-api-key=your-langsmith-api-key,Langsmith-Project=your-tracing-project-name"

`BG_JOB_ISOLATED_LOOPS`=true (execute background runs in an isolated event loop separate from the serving API event loop; Strand's OTel tracing contains synchronous code)